In [1]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from PIL import Image
import pandas as pd
from livelossplot import PlotLossesKeras
from sklearn.externals import joblib

/home/manish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/manish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
/home/manish/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/manish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def Densenet(input_shape,num_layers,filters,growth_rate,num_blocks,num_classes,compression,dropout_rate=None,weight_decay=1e-04):
    input_layer=Input(input_shape)
    net=Conv2D(filters=filters*2,kernel_size=7,strides=2,kernel_initializer='he_normal',padding='same',kernel_regularizer=regularizers.l2(weight_decay))(input_layer)
    net=MaxPool2D(pool_size=3,strides=2,padding='same')(net)
    for block in range(num_blocks-1):
        net,filters=dense_block(net,filters,num_layers[block],growth_rate,elu=True,dropout_rate=dropout_rate)
        net=transition_block(net,filters,compression,elu=True,dropout_rate=dropout_rate)
        filters=int(filters*compression)
    net,filters=dense_block(net,filters,num_layers[block],growth_rate,elu=True)
    net=GlobalAveragePooling2D()(net)
    fc=Dense(units=num_classes,activation="softmax")(net)
    model=Model(inputs=input_layer,outputs=fc)
    return model
def conv(inputs,filters,elu=True,dropout_rate=None):
    if not elu:
        x=BatchNormalization()(inputs)
        x=ReLU()(x)
        x=Conv2D(filters=filters*4,kernel_size=1,kernel_initializer="he_normal",strides=1,kernel_regularizer=regularizers.l2(weight_decay))(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        x=BatchNormalization()(x)
        x=ReLU()(x)
        x = ZeroPadding2D((1, 1))(x)
        x=Conv2D(filters=filters,kernel_size=3,kernel_initializer="he_normal",strides=1)(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
    else:
        x=Conv2D(filters=filters*4,kernel_size=1,kernel_initializer="he_normal",strides=1,activation='elu',kernel_regularizer=regularizers.l2(weight_decay))(inputs)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        x = ZeroPadding2D((1, 1))(x)
        x=Conv2D(filters=filters,kernel_size=3,kernel_initializer="he_normal",strides=1,activation='elu')(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
    return x

def dense_block(x,filters,num_layers,growth_rate,elu=True,dropout_rate=None):
    layer_list=[x]
    for i in range(num_layers):
        con=conv(x,growth_rate,elu,dropout_rate)
        layer_list.append(con)
        x=concatenate(layer_list,axis=-1)
#         x=concatenate([x,con],axis=-1)
        filters+=growth_rate
    return x,filters
        
def transition_block(inputs,filters,compression,elu=True,dropout_rate=None):
    if not elu:
        x=BatchNormalization()(inputs)
        x=ReLU()(x)
        x=Conv2D(filters=int(filters*compression),kernel_size=1,kernel_initializer="he_normal",strides=1,padding='same',kernel_regularizer=regularizers.l2(weight_decay))(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        x=AveragePooling2D(pool_size=2,strides=2)(x)
    else:
        x=Conv2D(filters=int(filters*compression),kernel_size=1,kernel_initializer="he_normal",strides=1,padding='same',kernel_regularizer=regularizers.l2(weight_decay))(inputs)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        x=AveragePooling2D(pool_size=2,strides=2)(x)
    return x
        

In [6]:
input_shape=(224,224,3)
num_layers=[6,12,24,16]
growth_rate=12
num_blocks=4
num_classes=5
filters=64
compression=0.5
dropout_rate=0
weight_decay=1e-04


In [7]:
model=Densenet(input_shape,num_layers,filters,growth_rate,num_blocks,num_classes,compression,dropout_rate,weight_decay)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_273 (Conv2D)             (None, 112, 112, 128 18944       input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 56, 56, 128)  0           conv2d_273[0][0]                 
__________________________________________________________________________________________________
conv2d_274 (Conv2D)             (None, 56, 56, 48)   6192        max_pooling2d_3[0][0]            
__________________________________________________________________________________________________
zero_paddi